In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Load and display the original image
img = cv2.imread('istockphoto-1212285302-612x612.jpg', 0)
img = cv2.resize(img, (150, 150))

In [ ]:
plt.axis('off')
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
print(f'image shape: {img.shape}', f'Number of genes:{img.size}')

In [ ]:
# Convert image to chromosome representation
image = np.asarray(img / 255, dtype=np.float16)
target_chromosome = image.flatten()

In [ ]:
# Define the fitness function
def fitness_fun(solution):
    fitness = np.sum(np.abs(target_chromosome - solution))
    fitness = np.sum(target_chromosome) - fitness
    return fitness

In [ ]:
def GA(num_generations, num_parents_mating, fitness_func, sol_per_pop, num_genes,
       init_range_low, init_range_high, mutation_percent_genes,
       random_mutation_min_val, random_mutation_max_val):

    population = initialize_population(sol_per_pop, num_genes, init_range_low, init_range_high)

    best_solution = None
    best_solution_fitness = -np.inf
    best_solution_generation = -1
    best_solution_array = np.empty(int(num_generations/100))
    index=0
    
    for generation in range(num_generations):
        fitness = np.array([fitness_func(solution) for solution in population])

        best_solution_idx = np.argmax(fitness)
        if fitness[best_solution_idx] > best_solution_fitness:
            best_solution = population[best_solution_idx]
            best_solution_fitness = fitness[best_solution_idx]
            best_solution_generation = generation

        parents = select_parents(population, fitness, num_parents_mating)

        offspring_crossover = crossover(parents, sol_per_pop - num_parents_mating)

        offspring_mutation = mutate(offspring_crossover, mutation_percent_genes,
                                    random_mutation_min_val, random_mutation_max_val)

        population[:num_parents_mating] = parents
        population[num_parents_mating:] = offspring_mutation
        
        if generation % 100 == 0:
            print("Generation:", generation, ", best_solution_fitness:", best_solution_fitness)
            best_solution_array[index]=best_solution_fitness
            index+=1

    return best_solution_array,best_solution, best_solution_fitness, best_solution_generation

def initialize_population(sol_per_pop, num_genes, init_range_low, init_range_high):
    population = np.random.uniform(low=init_range_low, high=init_range_high,
                                   size=(sol_per_pop, num_genes))
    return population

def select_parents(population, fitness, num_parents_mating):
    parents_indices = np.argsort(fitness)[-num_parents_mating:]
    parents = population[parents_indices]
    return parents

def crossover(parents, num_offspring):
    offspring = np.empty((num_offspring, parents.shape[1]))

    for idx in range(num_offspring):
        parent1_idx = idx % parents.shape[0]
        parent2_idx = (idx+1) % parents.shape[0]
        crossover_point = np.random.randint(1, parents.shape[1])
        offspring[idx] = np.concatenate((parents[parent1_idx][:crossover_point],
                                          parents[parent2_idx][crossover_point:]))

    return offspring

def mutate(offspring_crossover,mutation_percent_genes,
           random_mutation_min_val, random_mutation_max_val):
    mask = np.random.choice([True, False], size=offspring_crossover.shape,
                            p=[mutation_percent_genes, 1 - mutation_percent_genes])
    random_mutation = np.random.uniform(low=random_mutation_min_val,high=random_mutation_max_val,
                                                    size=offspring_crossover.shape)
    offspring_crossover[mask] = random_mutation[mask]
       

    return offspring_crossover


In [ ]:
# Define the GA parameters
num_generations = 30000
num_parents_mating = 1
sol_per_pop = 10
num_genes = image.size
init_range_low = 0.0
init_range_high = 1.0
mutation_percent_genes = 0.000095
random_mutation_min_val = 0.0
random_mutation_max_val = 1.0

# Run the GA
best_solution_array, best_solution, best_solution_fitness, best_solution_generation = GA(num_generations=num_generations,
                                                                     num_parents_mating=num_parents_mating,
                                                                     fitness_func=fitness_fun,
                                                                     sol_per_pop=sol_per_pop,
                                                                     num_genes=num_genes,
                                                                     init_range_low=init_range_low,
                                                                     init_range_high=init_range_high,
                                                                     mutation_percent_genes=mutation_percent_genes,
                                                                     
                                                                     random_mutation_min_val=random_mutation_min_val,
                                                                     random_mutation_max_val=random_mutation_max_val,
                                                                    )


In [ ]:
x=np.arange(0,num_generations,100)

plt.plot(x,best_solution_array)
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Fitness Progress')
plt.show()

print(f"Fitness value of the best solution = {best_solution_fitness}")
print(f"Best fitness value reached after {best_solution_generation} generations.")


In [ ]:
result = best_solution.reshape(150,150)
cv2.imwrite('Image_reconstruction.jpg',(result*255).astype(int))
image=cv2.imread('Image_reconstruction.jpg',0)
plt.axis('off')
plt.imshow(image,cmap='gray')
plt.title('Reproduction image')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
plt.title('original image')
plt.axis('off')
plt.imshow(img,cmap='gray')
plt.subplot(1,2,2)
plt.axis('off')
plt.title('Reproduction image')
plt.imshow(image,cmap='gray')
plt.show()